## Trained Embedded Layer
- created: 23 MArch 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Load Data 

In [14]:
def prep_data(data_path):

    # load data from preprocess pipeline
    dataset = np.load(join(data_path,'training_subsets.npz'))
    X_train =dataset['X_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    z_test = dataset['z_test']
    a_test = dataset['audio_test']
    a_val = dataset['audio_val']
    
    print('Train features and labels %s %s'%(str(X_train.shape),str(z_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(z_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(z_test.shape)))

    return X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val

In [15]:
DATA_PATH = '../../data/d_dataset_p2/prep_data_20'
X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val = prep_data(DATA_PATH)

Train features and labels (102624, 24, 60) (102624,)
Validating features and labels (12568, 24, 60) (12568,)
Test features and labels (12568, 24, 60) (12568,)


In [18]:
print(set(z_test))

{'@', 'E:', 'E', 'a', '9', '2', 'y', 'i', 'u', 'e', 'U', 'O', 'A', 'o'}


### One-hot label 

In [19]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

### Defined (Between) Embedding Model

In [12]:
PHONETIC_OUTPUT = len(set(z_train))
EMBEDDED_FEATURE = 128

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Flatten()(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [20]:
embedded = embeded_model(24, 60)
embedded.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 60)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 128)           96768     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)                43022     
Total params: 139,790
Trainable params: 139,790
Non-trainable params: 0
_________________________________________________________________


### Compile Model 

In [21]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [22]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [23]:
history = embedded.fit(X_train,z_train_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,z_val_oe),
    callbacks=callback_list)

Train on 102624 samples, validate on 12568 samples
Epoch 1/100
102624/102624 - 11s - loss: 1.8974 - accuracy: 0.8717 - val_loss: 1.8110 - val_accuracy: 0.9465
Epoch 2/100
102624/102624 - 5s - loss: 1.8275 - accuracy: 0.9304 - val_loss: 1.8000 - val_accuracy: 0.9564
Epoch 3/100
102624/102624 - 5s - loss: 1.8138 - accuracy: 0.9434 - val_loss: 1.7960 - val_accuracy: 0.9597
Epoch 4/100
102624/102624 - 4s - loss: 1.8053 - accuracy: 0.9509 - val_loss: 1.7951 - val_accuracy: 0.9606
Epoch 5/100
102624/102624 - 4s - loss: 1.7998 - accuracy: 0.9565 - val_loss: 1.7921 - val_accuracy: 0.9632
Epoch 6/100
102624/102624 - 5s - loss: 1.7956 - accuracy: 0.9603 - val_loss: 1.7983 - val_accuracy: 0.9571
Epoch 7/100
102624/102624 - 5s - loss: 1.7932 - accuracy: 0.9626 - val_loss: 1.7895 - val_accuracy: 0.9659
Epoch 8/100
102624/102624 - 5s - loss: 1.7907 - accuracy: 0.9652 - val_loss: 1.7920 - val_accuracy: 0.9629
Epoch 9/100
102624/102624 - 5s - loss: 1.7902 - accuracy: 0.9655 - val_loss: 1.7897 - val_ac

In [24]:
embedded.save('between_embedded.hdf5')

In [26]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
#     print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [27]:
z_pred = embedded.predict(X_test)
evaluate_model(z_pred, z_test_oe)

Classification Report
              precision    recall  f1-score   support

           2       0.96      0.99      0.97       934
           9       0.99      0.99      0.99       886
           @       0.98      0.99      0.98       903
           A       0.94      0.98      0.96       888
           E       0.97      0.97      0.97       890
          E:       0.98      0.96      0.97       879
           O       0.97      0.98      0.97       964
           U       0.97      0.94      0.95       890
           a       0.98      0.94      0.96       942
           e       0.96      0.98      0.97       894
           i       0.99      0.97      0.98       903
           o       0.96      0.92      0.94       835
           u       0.92      0.96      0.94       895
           y       0.98      0.96      0.97       865

    accuracy                           0.97     12568
   macro avg       0.97      0.97      0.97     12568
weighted avg       0.97      0.97      0.97     12568

